In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# import the yaml libraries for configuration files
import yaml

In [12]:
# read the data from config.yml (only get the first model for now, but support for many batched models shouldn't be difficult to implement)
with open('config.yml', 'r') as f:
    data = yaml.load(f, Loader=yaml.SafeLoader)

rootDir = data.get('root_path') # get the root path to all model folders from config.yml

models = data["models"]

modelName = models[0].get("data").get("name")
residualLimbName = models[0].get("data").get('residualLimbName')
distalProstheticName = models[0].get("data").get('distalProstheticName')

# read the excel data from dirs specified by config.yml
muscleMags = pd.read_excel(rootDir + modelName + "/FEA/OpenSim data/muscleMags.xlsx")
muscleVecs = pd.read_excel(rootDir + modelName + "/FEA/OpenSim data/muscleVecs.xlsx")
jointMags = pd.read_excel(rootDir + modelName + "/FEA/OpenSim data/jointMags.xlsx")

In [13]:
residualLimbName2 = "_on_" + residualLimbName

In [14]:
decimals = 3
muscleMags['time'] = muscleMags['time'].apply(lambda x: round(x, decimals))
muscleVecs['time'] = muscleVecs['time'].apply(lambda x: round(x, decimals))
jointMags['time'] = jointMags['time'].apply(lambda x: round(x, decimals))

In [15]:
muscleMags = muscleMags.drop_duplicates(subset = ['time'], keep = 'first')
muscleVecs = muscleVecs.drop_duplicates(subset = ['time'], keep = 'first')
jointMags = jointMags.drop_duplicates(subset = ['time'], keep = 'first')

In [16]:
data = muscleMags.merge(muscleVecs, on = 'time')
data = data.merge(jointMags, on = 'time')

In [17]:
muscles = []
for column in muscleVecs.columns:
    if residualLimbName2 in column:
        if column[:-14] not in muscles:
            muscles.append(column[:-14])
muscle_names = muscles

In [18]:
# create muscle force magnitudes
x = len(data[muscle_names[0]])
y = len(muscle_names)
muscle_forces = np.zeros((x, y))
for i in range(len(muscle_names)):
    name = muscle_names[i]
    f = list(data[name])
    muscle_forces[:,i] = f

In [19]:
# create muscle force vectors
coords = ['X2', 'Y2', 'Z2']
name = muscle_names[0]
namet = name + '_' + coords[0] + residualLimbName2
x = len(list(data[name]))
y = len(muscle_names)
muscle_vectors = np.zeros((3, x, y))
for i in range(len(muscle_names)):
    name = muscle_names[i]
    vec = [[], [], []]
    for j in range(len(coords)):
        namet = name + '_' + coords[j] + residualLimbName2
        vec[j] = list(data[namet])
    muscle_vectors[:,:,i] = vec

In [20]:
outputFileName = models[0].get("data").get("outputFileName")

# open and begin writing to the python file specified in config.yml
fout = open(rootDir + outputFileName, 'w')

i = 28
forces = list(muscle_forces[:,i])
forces = [i if i != 0 else 0.000001 for i in forces]

fout.write(str(muscle_names[i]) + "\n")
fout.write(str(forces) + "\n")
fout.write(str(muscle_vectors[:,:,i].tolist()) + "\n")

1856

In [21]:
fout.write("# create muscle force magnitudes\n")

for i in range(len(muscle_names)):
    forces = list(muscle_forces[:,i])
    forces = [j if j != 0 else 0.000001 for j in forces]
    fout.write(f"{muscle_names[i]} = {forces}" + "\n")

muscleNamesString = f"muscle_forces = np.array({muscle_names})".replace("'", "")
fout.write(muscleNamesString + "\n\n# Create muscle force vectors\n")

for i in range(len(muscle_names)):
    fout.write(f"{muscle_names[i]} = {muscle_vectors[:,:,i].tolist()}" + "\n")

muscleNamesString = f"muscle_forces = np.array({muscle_names})".replace("'", "")
fout.write(muscleNamesString + "\n\n" + f"muscle_names = {muscle_names}" + "\n\n" + "# create joint force magnitudes\n")

fx = f"{distalProstheticName}_on_{residualLimbName}_in_{residualLimbName}_fx"
fy = f"{distalProstheticName}_on_{residualLimbName}_in_{residualLimbName}_fy"
fz = f"{distalProstheticName}_on_{residualLimbName}_in_{residualLimbName}_fz"

fout.write(f"kneeLoad_x = {list(data[fx])}" + "\n")
fout.write(f"kneeLoad_y = {list(data[fy])}" + "\n")
fout.write(f"kneeLoad_z = {list(data[fz])}" + "\n")

fout.close()

KeyError: 'walker_knee_r_on_femur_r_in_femur_r_fx'